In [15]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as lin
import numpy.random as rnd

def fbSubs(LU, b):
    (rows, columns) = LU.shape

    # solve LZ = b
    z = np.full(rows, 0, dtype=np.number)
    for row in range(rows):
        sub = np.dot(LU[row], z)
        z[row] = b[row] - sub

    # solve RX = Z
    x = np.full(rows, 0, dtype=np.number)
    for row in reversed(range(rows)):
        sub = np.dot(LU[row], x)
        x[row] = (z[row] - sub)/LU[row, row]

    return x

def LU(A):
    (rows, columns) = A.shape

    # decompose inplace into LU matrix
    for column in range(columns):
        for row in range(column + 1, columns):
            scalar = A[row, column] / A[column, column]
            A[row, column:columns] -= (A[column] * scalar)[column:columns]
            A[row, column] = scalar

    return A

def linsolve(A, b):
    lu = LU(A)
    x = fbSubs(lu, b)
    return x


# random orthogonal matrix
def rndOrtho(n):
    S = rnd.rand(n,n)
    S = S - S.T
    O = lin.solve(S - np.identity(n), S + np.identity(n))
    return O

# random matrix with specified condition number
def rndCond(n, cond):
    d = np.logspace(-np.log10(cond)/2, np.log10(cond)/2,n)
    A = np.diag(d)
    U,V = rndOrtho(n), rndOrtho(n)
    return U@A@V.T


# Tests

In [16]:
n = 6 # size of test matrix

# test fbSubs
for k in range(1000):                      # Testläufe
    Lu = np.array( np.random.rand(n,n) )   # zufällige Matrix LR
    rhs = np.array(np.random.rand(n))    # zufällige rechte Seite des LGS
    x = fbSubs(Lu, rhs)   # Aufruf der eigenen Funktion

    L,R = np.tril(Lu,-1)+np.identity(n), np.triu(Lu) # L und R extrahieren

    assert( np.linalg.norm(rhs - L@R@x) < 1e-10)     # Test, mit numerischer Toleranz

# test LU
for k in range(1000):
    A = np.array( np.random.rand(n,n) )   # zufällige Matrix A erzeugen
    Lu = LU(A.copy())                  # LU-Zerlegung von A
    L,R = np.tril(Lu,-1)+np.identity(n), np.triu(Lu)  # Matrizen L, R extrahieren
    assert( np.linalg.norm(L@R - A) < 1e-8)

# test linsolve
for k in range(1000):
    A = np.random.rand(n,n)
    rhs = np.random.rand(n)
    x = linsolve(A.copy(), rhs)
    assert( np.linalg.norm(rhs - A @ x) < 1e-10)

for k in range(1000):
    A = rndCond(n, 1e14)
    # code